# Data verzamelen
Ik heb 2 databases met data. Een is een sqlite3 bestand van [Podcast Index](https://podcastindex.org/). De ander is een het resultaat van het scrapen van de pagina's van de categorieën op [iTunes](https://podcasts.apple.com/us/genre/podcasts/id26) 

## SQL manipulatie
Hieronder de code gebruikt om het SQL bestand aan te passen naar mysql

sqlite -> mysql via [sqlite3mysql](https://pypi.org/project/sqlite3-to-mysql/)

In [ ]:
pip install qlite3-to-mysql
# > sqlite3mysql -f C:\Users\Jeanny\Projects\bit-academy\final_project\podcastindex_feeds.db -d podcast_feeds -u root    

Voeg een index toe aan kolom itunesId om <code>JOIN</code> met table <code>categorieën</code> sneller te maken

In [ ]:
add_fk = '''
    ALTER TABLE podcasts ADD FORGEIN KEY (itunesId);
'''

## Web scraping
Hieronder de uitleg van de helpers om een database van podcast categorieën te maken. 

1. Nieuwe tabel categorieën maken

Als eerste kopieerde ik de kolom itunesId van table podcasts. Dit probeerde ik eerst via SQL, maar ik kreeg telkens timeouts. De hack: sql -> csv -> sql. LOAD DATA is veel efficiënter dan kopiëren.

In [ ]:
sql_to_csv = '''
    USE DATABASE podcast_feeds;
    SELECT DISTINCT itunesId FROM podcasts WHERE itunesId > 0
    INTO OUTFILE "C:\\Users\\Jeanny\\Projects\\bit-academy\\final_project\\itunedId.csv"
    FIELDS ENCLOSED BY '"'
    TERMINATED BY ';'
    ESCAPED BY '"'
    LINES TERMINATED BY '\r\n';
'''

new_table = '''
    CREATE TABLE `categories` (`itunesId` INT(11));
    ALTER TABLE `categories`
    ADD PRIMARY KEY (`itunesId`);
'''

Deze data kan dan weer in de nieuwe tabel geimporteerd worden via:
    phpMyAdmin -> import CSV using LOAD DATA, seperaed by ",", column name= itunesID.
    
iTunes heeft 110(!) categorieën, da's iets te veel typewerk. Hiervoor heb ik <code>helpers/create_categories_table.py</code> gebruikt. Dit programma maakt een lijst van alle categorieën op de itunes podcast pagina, en voegt voor iedere categorie een kolom toe:

In [ ]:
new_column = '''
    f"ALTER TABLE `categories` ADD COLUMN `{category}` BOOLEAN DEFAULT false;"
'''

De kolommen hebben allemaal een boolean value. Of de podcast heeft de categorie wel (True = 1) of niet (False = 0). Er wordt nu False neergezet, wat na het scrapen geupdate kan worden.

2. Scrapen van categorieën

Zie <code>helpers/itunes_crawler.py</code>. De logica is als volgt: 
- Maak een lijst van alle categorieën
- Voor elke categorie:
    - Crawl alle pagina's van categorie. (A-Z, meerdere pagina's per letter)
    - Voor iedere link op een pagina, voeg de itunesId toe aan een lijst.
    - Na het crawlen is er een lijst met alle itunesID voor een categorie
    - Verbind met mySQL, update TABLE voor de categorie, waar itunesID table = itunesID lijst False -> True